## 1

In [3]:
!pip install dowhy

     |████████████████████████████████| 152 kB 30.2 MB/s 
  Attempting uninstall: pydot
    Found existing installation: pydot 1.3.0
    Uninstalling pydot-1.3.0:
      Successfully uninstalled pydot-1.3.0


In [7]:
import sys
sys.path.append('dowhy')
import random
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LogisticRegression

In [29]:
data= pd.read_csv("https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv", header = None)
col =  ["t", "y_factual", "y_cfactual", "mu0", "mu1" ,]

for i in range(1,26):
    col.append("x"+str(i))
data.columns = col
data.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0


In [28]:
data.info

<bound method DataFrame.info of                      0                   1                   2   \
0                    x1                  x2                  x3   
1      1.39739503392468   0.996346250402707   -1.10562394853556   
2     0.269032653832557   0.196818122274482   0.383827971313067   
3      1.05153683313016    1.79587437853093   -1.10562394853556   
4     0.662446357236323   0.196818122274482  -0.733260968573404   
..                  ...                 ...                 ...   
743    -1.8536720535438   -2.20176626211019    1.50091691119954   
744   -1.06684464673627   -1.40223813398197    1.12855393123738   
745   -1.04522850918662   -1.33727610641504    1.12855393123738   
746  -0.115734594551345  -0.602710005853743   -1.10562394853556   
747    1.41901117147433    1.39611031446682  -0.733260968573404   

                     3                   4                   5    6    7   \
0                    x4                  x5                  x6   x7   x8   
1    -0.8

In [24]:
data.describe

<bound method NDFrame.describe of                       t                  yf                 ycf  \
0                    x1                  x2                  x3   
1      1.39739503392468   0.996346250402707   -1.10562394853556   
2     0.269032653832557   0.196818122274482   0.383827971313067   
3      1.05153683313016    1.79587437853093   -1.10562394853556   
4     0.662446357236323   0.196818122274482  -0.733260968573404   
..                  ...                 ...                 ...   
743    -1.8536720535438   -2.20176626211019    1.50091691119954   
744   -1.06684464673627   -1.40223813398197    1.12855393123738   
745   -1.04522850918662   -1.33727610641504    1.12855393123738   
746  -0.115734594551345  -0.602710005853743   -1.10562394853556   
747    1.41901117147433    1.39611031446682  -0.733260968573404   

                    mu0                 mu1                  x1   x2   x3  \
0                    x4                  x5                  x6   x7   x8   
1    -0

In [30]:
from dowhy import CausalModel

In [31]:
xs = ""
for i in range(1,26):
    xs += ("x"+str(i)+"+")

model=CausalModel(
        data = data,
        treatment='treatment',
        outcome='y_factual',
        common_causes=xs.split('+')
        )


## 2

In [32]:
identified_estimand = model.identify_effect()

In [33]:

# Estimate the causal effect and compare it with Average Treatment Effect
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", test_significance=True
)

print(estimate)

print("Causal Estimate is " + str(estimate.value))
data_1 = data[data["treatment"]==1]
data_0 = data[data["treatment"]==0]

print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))

Estimation failed! No relevant identified estimand available for this estimation method.
Causal Estimate is None
ATE 4.021121012430832


## 3

In [34]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_matching"
)

print("Causal Estimate is " + str(estimate.value))

print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))


Causal Estimate is None
ATE 4.021121012430832


## 4

In [35]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification", method_params={'num_strata':50, 'clipping_threshold':5}
)

print("Causal Estimate is " + str(estimate.value))
print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))

Causal Estimate is None
ATE 4.021121012430832


## 5

In [44]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_weighting"
)

print("Causal Estimate is " + str(estimate.value))

print("ATE", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))

Causal Estimate is None
ATE 4.021121012430832


In [43]:

refute_results=model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause")
print(refute_results)

ValueError: ignored

In [38]:
res_placebo=model.refute_estimate(identified_estimand, estimate,
        method_name="placebo_treatment_refuter", placebo_type="permute")
print(res_placebo)

ValueError: ignored

In [41]:

res_subset=model.refute_estimate(identified_estimand, estimate,
        method_name="data_subset_refuter", subset_fraction=0.9)
print(res_subset)

ValueError: ignored

## 1 JOBS

In [45]:
data1= pd.read_csv("/content/jobs.csv", header = None)
col =  ["t", "y", "e", "mu0", "mu1" ,]

for i in range(1,26):
    col.append("x"+str(i))
data.columns = col
data.head()

,t,y,e,mu0,mu1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0


In [46]:
xs1 = ""
for i in range(1,26):
    xs += ("x"+str(i)+"+")

model=CausalModel(
        data = data,
        treatment='t',
        outcome='y',
        common_causes=xs.split('+')
        )

In [48]:
identified_estimand1 = model.identify_effect()

## 2 JOBS

In [49]:
estimate1 = model.estimate_effect(identified_estimand1,
        method_name="backdoor.linear_regression", test_significance=True
)

print(estimate1)

print("Causal Estimate is " + str(estimate1.value))
data_1 = data[data["t"]==1]
data_0 = data[data["t"]==0]

print("ATE", np.mean(data_1["y"])- np.mean(data_0["y"]))

Estimation failed! No relevant identified estimand available for this estimation method.
Causal Estimate is None
ATE 4.021121012430832


## 3 JOBS

In [51]:
estimate1 = model.estimate_effect(identified_estimand1,
        method_name="backdoor.propensity_score_matching"
)

print("Causal Estimate is " + str(estimate1.value))

print("ATE", np.mean(data_1["y"])- np.mean(data_0["y"]))


Causal Estimate is None
ATE 4.021121012430832


## 4

In [52]:

estimate1 = model.estimate_effect(identified_estimand1,
        method_name="backdoor.propensity_score_stratification", method_params={'num_strata':50, 'clipping_threshold':5}
)

print("Causal Estimate is " + str(estimate1.value))
print("ATE", np.mean(data_1["y"])- np.mean(data_0["y"]))

Causal Estimate is None
ATE 4.021121012430832


## 5

In [53]:
estimate1 = model.estimate_effect(identified_estimand1,
        method_name="backdoor.propensity_score_weighting"
)

print("Causal Estimate is " + str(estimate1.value))

print("ATE", np.mean(data_1["y"])- np.mean(data_0["y"]))

Causal Estimate is None
ATE 4.021121012430832


In [54]:
refute_results1 =model.refute_estimate(identified_estimand1, estimate1,
        method_name="random_common_cause")
print(refute_results1)

ValueError: ignored

In [ ]:
res_placebo1 =model.refute_estimate(identified_estimand1, estimate1,
        method_name="placebo_treatment_refuter", placebo_type="permute")
print(res_placebo1)